In [31]:
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm


import keras_tuner
import pandas as pd
import numpy as np
import os
import pickle
import matplotlib.pyplot as plt

## Data loading and preprocessing

In [32]:
datapath = r'/Users/ignacioaranguren/QR_assignment/'
os.chdir(datapath)

In [33]:
columns=['Matching Time', 'Receiving Time', 'Symbol']
for i in range(1,11):
    columns += [f'BID_PRICE{i}', f'BID_QTY_{i}', f'ASK_PRICE_{i}', f'ASK_QTY_{i}']

In [34]:
df_train_tmp = pd.read_csv(f'data/exchange1_20210519.csv', names=columns).set_index('Matching Time').append(pd.read_csv(f'data/exchange1_20210520.csv', names=columns).set_index('Matching Time'))
df_test_tmp = pd.read_csv(f'data/exchange1_20210521.csv', names=columns).set_index('Matching Time')

In [35]:
df_train_tmp['MID_PRICE'] = (df_train_tmp['BID_PRICE1'] + df_train_tmp['ASK_PRICE_1']) / 2
df_test_tmp['MID_PRICE'] = (df_test_tmp['BID_PRICE1'] + df_test_tmp['ASK_PRICE_1']) / 2

In [36]:
def mid_price_trend(df, k, alpha):
    # Contiguous mid values may be noisy. We need to smooth them by calculating the mean to infere the price trend
    trend = []
    for i in tqdm(range(k//2, len(df['MID_PRICE']) - k//2)):
        prev_k_mid_prices = df.iloc[i - k//2: i, -1].mean()
        next_k_mid_prices = df.iloc[i: i + k//2, -1].mean()
        if prev_k_mid_prices < next_k_mid_prices * (1 + alpha):
            trend += [1]
        elif prev_k_mid_prices > next_k_mid_prices * (1 - alpha):
            trend += [-1]
        else:
            trend += [0]
    return trend

In [37]:
K_HORIZON = 20
ALPHA = 0

In [38]:
trend_train = mid_price_trend(df_train_tmp, K_HORIZON, ALPHA)
df_train_tmp['TREND_MID_PRICE'] = pd.Series(trend_train, index=df_train_tmp.index[K_HORIZON//2:-K_HORIZON//2]) 
df_train_tmp = df_train_tmp.dropna()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4523864/4523864 [20:08<00:00, 3743.10it/s]


In [39]:
trend_test = mid_price_trend(df_test_tmp, K_HORIZON, ALPHA)
df_test_tmp['TREND_MID_PRICE'] = pd.Series(data=trend_test, index=df_test_tmp.index[K_HORIZON//2:-K_HORIZON//2]) 
df_test_tmp = df_test_tmp.dropna()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2403150/2403150 [10:55<00:00, 3668.28it/s]


In [40]:
df_train_tmp.describe()

,BID_PRICE1,BID_QTY_1,ASK_PRICE_1,ASK_QTY_1,BID_PRICE2,BID_QTY_2,ASK_PRICE_2,ASK_QTY_2,BID_PRICE3,BID_QTY_3,...,BID_PRICE9,BID_QTY_9,ASK_PRICE_9,ASK_QTY_9,BID_PRICE10,BID_QTY_10,ASK_PRICE_10,ASK_QTY_10,MID_PRICE,TREND_MID_PRICE
count,4.523864e+06,4.523864e+06,4.523864e+06,4.523864e+06,4.523864e+06,4.523864e+06,4.523864e+06,4.523864e+06,4.523864e+06,4.523864e+06,...,4.523864e+06,4.523864e+06,4.523864e+06,4.523864e+06,4.523864e+06,4.523864e+06,4.523864e+06,4.523864e+06,4.523864e+06,4.523864e+06
mean,3.963470e+04,1.356158e+00,3.963820e+04,1.235090e+00,3.963142e+04,1.087778e+00,3.964151e+04,9.648139e-01,3.962892e+04,1.131413e+00,...,3.961742e+04,1.398526e+00,3.965576e+04,1.333236e+00,3.961571e+04,1.453538e+00,3.965749e+04,1.375663e+00,3.963645e+04,-9.600863e-03
std,1.577665e+03,6.476020e+00,1.575651e+03,6.119528e+00,1.578698e+03,4.793466e+00,1.574545e+03,4.609856e+00,1.579507e+03,4.653702e+00,...,1.583794e+03,4.713733e+00,1.568959e+03,4.927722e+00,1.584524e+03,4.728163e+00,1.568183e+03,4.891516e+00,1.576654e+03,9.304395e-01
min,2.910000e+04,1.000000e-04,2.918900e+04,1.000000e-04,2.909500e+04,1.000000e-04,2.921300e+04,1.000000e-04,2.908600e+04,1.000000e-04,...,2.900000e+04,1.000000e-04,2.936500e+04,1.000000e-04,2.897800e+04,1.000000e-04,2.945900e+04,1.000000e-04,2.914450e+04,-1.000000e+00
25%,3.899000e+04,4.830000e-02,3.899400e+04,4.810000e-02,3.898700e+04,4.000000e-02,3.899700e+04,4.000000e-02,3.898500e+04,5.000000e-02,...,3.897400e+04,5.550000e-02,3.901000e+04,5.870000e-02,3.897200e+04,5.680000e-02,3.901200e+04,5.880000e-02,3.899250e+04,-1.000000e+00
50%,3.978500e+04,1.750000e-01,3.978800e+04,1.753000e-01,3.978200e+04,1.270000e-01,3.979100e+04,1.275000e-01,3.978000e+04,1.342000e-01,...,3.976900e+04,2.918000e-01,3.980300e+04,3.100000e-01,3.976800e+04,3.200000e-01,3.980500e+04,3.533000e-01,3.978650e+04,0.000000e+00
75%,4.048000e+04,5.799000e-01,4.048300e+04,5.736000e-01,4.047700e+04,4.114000e-01,4.048600e+04,4.099000e-01,4.047500e+04,4.903000e-01,...,4.046500e+04,1.024300e+00,4.049800e+04,1.084900e+00,4.046400e+04,1.124000e+00,4.049900e+04,1.200000e+00,4.048150e+04,1.000000e+00
max,4.364300e+04,7.938822e+02,4.365000e+04,5.456851e+02,4.364200e+04,7.938773e+02,4.365200e+04,5.456851e+02,4.364000e+04,7.845476e+02,...,4.362900e+04,8.020410e+02,4.366000e+04,5.466064e+02,4.362600e+04,8.019155e+02,4.366100e+04,5.466064e+02,4.364650e+04,1.000000e+00


In [41]:
df_test_tmp.describe()

,BID_PRICE1,BID_QTY_1,ASK_PRICE_1,ASK_QTY_1,BID_PRICE2,BID_QTY_2,ASK_PRICE_2,ASK_QTY_2,BID_PRICE3,BID_QTY_3,...,BID_PRICE9,BID_QTY_9,ASK_PRICE_9,ASK_QTY_9,BID_PRICE10,BID_QTY_10,ASK_PRICE_10,ASK_QTY_10,MID_PRICE,TREND_MID_PRICE
count,2.403150e+06,2.403150e+06,2.403150e+06,2.403150e+06,2.403150e+06,2.403150e+06,2.403150e+06,2.403150e+06,2.403150e+06,2.403150e+06,...,2.403150e+06,2.403150e+06,2.403150e+06,2.403150e+06,2.403150e+06,2.403150e+06,2.403150e+06,2.403150e+06,2.403150e+06,2.403150e+06
mean,3.911964e+04,9.925812e-01,3.912242e+04,9.672171e-01,3.911676e+04,6.605518e-01,3.912533e+04,6.149208e-01,3.911458e+04,7.417045e-01,...,3.910476e+04,1.181195e+00,3.913760e+04,1.051765e+00,3.910333e+04,1.256497e+00,3.913909e+04,1.113652e+00,3.912103e+04,-7.944157e-03
std,2.025563e+03,4.802817e+00,2.024988e+03,6.709653e+00,2.025811e+03,3.275450e+00,2.024741e+03,4.627409e+00,2.026018e+03,3.489606e+00,...,2.027051e+03,4.093494e+00,2.023451e+03,4.826758e+00,2.027198e+03,4.199408e+00,2.023295e+03,4.706735e+00,2.025275e+03,9.030111e-01
min,3.350400e+04,1.000000e-04,3.351800e+04,1.000000e-04,3.349100e+04,1.000000e-04,3.352500e+04,1.000000e-04,3.348400e+04,1.000000e-04,...,3.347600e+04,1.000000e-04,3.354500e+04,1.000000e-04,3.347200e+04,1.000000e-04,3.354600e+04,1.000000e-04,3.351700e+04,-1.000000e+00
25%,3.706900e+04,6.250000e-02,3.707200e+04,6.020000e-02,3.706600e+04,4.200000e-02,3.707600e+04,4.280000e-02,3.706400e+04,5.000000e-02,...,3.705300e+04,5.100000e-02,3.708800e+04,5.000000e-02,3.705200e+04,5.160000e-02,3.709000e+04,5.030000e-02,3.707050e+04,-1.000000e+00
50%,3.991800e+04,2.147000e-01,3.992100e+04,2.017000e-01,3.991600e+04,1.303000e-01,3.992300e+04,1.339000e-01,3.991400e+04,1.365000e-01,...,3.990500e+04,3.210000e-01,3.993500e+04,3.180000e-01,3.990400e+04,3.508000e-01,3.993600e+04,3.430000e-01,3.991950e+04,0.000000e+00
75%,4.086500e+04,6.661000e-01,4.086700e+04,6.010000e-01,4.086200e+04,3.992000e-01,4.087100e+04,3.813000e-01,4.086000e+04,4.449000e-01,...,4.085200e+04,9.997000e-01,4.088100e+04,9.748000e-01,4.085100e+04,1.037800e+00,4.088300e+04,1.012600e+00,4.086550e+04,1.000000e+00
max,4.232200e+04,2.988223e+02,4.233500e+04,3.982194e+02,4.231100e+04,3.293249e+02,4.234800e+04,3.729444e+02,4.230900e+04,3.293249e+02,...,4.229600e+04,3.298676e+02,4.239700e+04,3.730994e+02,4.229200e+04,3.297231e+02,4.239900e+04,3.729886e+02,4.232850e+04,1.000000e+00


In [42]:
df_test_tmp.to_csv('data/sets/unnormalized/test_set_k_20.csv',columns=list(df_test_tmp.columns.values))
df_train_tmp.to_csv('data/sets/unnormalized/train_set_k_20.csv',columns=list(df_train_tmp.columns.values))